In [1]:
#################################
# filenames
#################################

number = 38

probs_file = 'newprobs'+str(number)+'.json'
filename3s = 'newprobs'+str(number)+'genre.json'
filename3se = 'newprobs'+str(number)+'e.txt'
histo_file = 'histo'+str(number)+'.txt'
url_dict_file = 'url.json'


##############################
# setup
##############################

import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import random

tstart = time.time()
print(tstart)

api_url = 'https://api.hooktheory.com/v1/'
login = {"username": "jl8429",
         "password": ""}

r = requests.post(api_url+'users/auth', data=login)
#print('r', r)
#print(r.json())
'''
{'id': 136206, 'username': 'myUsername', 'activkey': 'myKey'}
'''

activkey = '0e11e1d65a021d0a883f309f813d74ad'
header = {"Authorization": "Bearer "+activkey}


#################################
# RELOAD url_dict
#################################

with open(url_dict_file, 'r') as f:
    url_dict = json.load(f)

big_dict = {}


#################################
# continue
#################################

#with open('chordProbs3new.json', "r") as f:
with open(probs_file, "r") as f:
    data3 = json.load(f)
f2 = open(filename3se, 'w')

j = 1
totals = str(len(data3))
print(totals)

histo = [0]*5000

driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")

for d in data3:
    chord1, chord2, chord3, chord4 = d['child_path'].split(",")
    mykey = str(chord1)+','+str(chord2)+','+str(chord3)+','+str(chord4)
    print(str(j)+'/'+totals, mykey)
    song_list = []
    
    # don't overwrite what we've done already
    if not big_dict.get(mykey, False):
        try:
            myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
            
            # slow down...
            time.sleep(1)
            if str(myData)=="<Response [429]>":
                #print("too many requests")
                time.sleep(11)
                myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
                if str(myData)=="<Response [429]>":
                    time.sleep(11)
                    print("too many requests")
            
            page = 1
            keepGoing = True
            #while myData.content and myData.content != b'No songs match this chord progressionnull':
            while keepGoing:
                # can't parse if it's this byte string
                if myData.content != b'No songs match this chord progressionnull':
                    #print('page', page)
                    # parse each song
                    try:
                        my_json = myData.json()
                        songs = len(my_json)
                        #print('songs len', songs)
                        if songs:
                            # for each song, pull the genre, add it to the url_dict and big_dict, then write it to the file
                            for i in range(songs):
                                url = my_json[i]['url'].split("#")[0]
                                #print(url)
                                # check if already in url_dict to save us from having to load the page slowly and parse
                                if url in url_dict:
                                    #print(url, url_dict[url])
                                    my_json[i]['genre'] = url_dict[url]
                                # if not in dict, we need to parse page and add to dict
                                else:
                                    #time.sleep(1)
                                    driver.get(url)
                                    html = driver.page_source
                                    soup = BeautifulSoup(html, 'html.parser')
                                    try:
                                        #genre = soup.find_all(['multiselect'])[0].find('a').contents[0]
                                        # some songs don't have a genre and some have multiple genres...
                                        genres = soup.find_all(['multiselect'])[0].find_all('a')
                                        gk = 0
                                        glist = []
                                        for g in genres:
                                            # .find_all() will return a lot of garbage, but the genres will have a length of 1
                                            if len(g) > 1:
                                                # if even the first is garbage, it has no genre
                                                if gk == 0:
                                                    my_json[i]['genre'] = ["Unknown"]
                                                    url_dict[url] = ["Unknown"]
                                                break
                                            else:
                                                glist.append(g.contents[0])
                                            gk += 1
                                        my_json[i]['genre'] = glist
                                        url_dict[url] = glist
                                    except:
                                        print('3. Unable to extract genre for', url)
                                        my_json[i]['genre'] = ["Unknown"]
                                        url_dict[url] = ["Unknown"]

                                #driver.close()
                                # clean up and append song to list
                                my_json[i].pop('section', None)
                                my_json[i]['url'] = url
                                #f.write('\n'+str(my_json[i])+',')
                                #print(my_json[i])
                                song_list.append(my_json[i])

                            if songs == 20:
                                #keepGoing = True
                                page += 1
                                myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                # slow down...
                                time.sleep(1)
                                if str(myData)=="<Response [429]>":
                                    #print("too many requests")
                                    time.sleep(11)
                                    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                    if str(myData)=="<Response [429]>":
                                        time.sleep(11)
                                        print("too many requests")
                            else:
                                # update histogram of how many songs per chord progression
                                index = page*20 - 20 + songs
                                print('histo1', index)
                                histo[index] += 1
                                keepGoing = False
                        else:
                            # update histogram of how many songs per chord progression
                            index = page*20 - 20
                            print("histo2", index)
                            histo[index] += 1
                            keepGoing = False

                    except:
                        # for some reason, page=20 doesn't work on 1-5-6-4... there are other data corruptions too
                        print('2. Unable to extract for', chord1, chord2, chord3, chord4, "page", page)
                        f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
                        f2.close()
                        f2 = open(filename3se, 'a')

                        page += 1
                        myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                        # slow down...
                        time.sleep(1)
                        if str(myData)=="<Response [429]>":
                            #print("too many requests")
                            time.sleep(11)
                            myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                            if str(myData)=="<Response [429]>":
                                time.sleep(11)
                                print("too many requests")
                else:
                    print("No songs match this chord progression", mykey)
                    keepGoing = False

        except:
            print('1. Unable to extract for', chord1, chord2, chord3, chord4)
            f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
            f2.close()
            f2 = open(filename3se, 'a')

            # slow down
            time.sleep(1)
        
        big_dict[mykey] = song_list
    #f.close()
    #f = open(filename3s, 'a')
    #time.sleep(1)
    j += 1
    # restart the driver every 100 chord progressions
    if j%100==0:
        driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")
    
f2.close()


with open(filename3s, "w") as f:
    json.dump(big_dict, f)

tend = time.time()
print(tend)
#f.close()
print(tend-tstart)

print(histo[:101])
with open(histo_file, "w") as f:  
    f.write(str(histo))

with open(url_dict_file, "w") as f:
    json.dump(url_dict, f)

1615597646.0925322
500
1/500 4,2,5,542
histo1 2
2/500 6,642,4,L4
histo1 1
3/500 46,1,16,5
histo1 4
4/500 27,Y27,665,5
histo1 1
5/500 464,56,5,4
histo1 1
6/500 5/5,67,1,3
histo1 1
7/500 6,27,2,16
histo1 1
8/500 464,4,3,564
histo1 1
9/500 2,3,6,365
histo1 1
10/500 5,27,1,4
histo1 1
11/500 5,D3,4,1
histo1 1
12/500 5/2,1,5/5,3
histo1 1
13/500 6,4,1,3
histo1 39
14/500 7/6,6,4,365
histo1 1
15/500 56,66,16,4
histo1 2
16/500 1,27,67,4
histo1 1
17/500 b7,5,2,5
histo1 1
18/500 6,17,4,36
histo1 1
19/500 46,5,56,6
histo1 1
20/500 b3,1,5,b7
histo1 1
21/500 77,7,664,57/6
histo1 1
22/500 164,5,4,2
histo1 1
23/500 56,5,1,6
histo1 1
24/500 5,6,56,3
histo1 1
25/500 47,5,6,3
histo1 4
26/500 1,27,6,5
histo1 6
27/500 665,164,b1,164
histo1 1
28/500 46,1,564,6
histo1 1
29/500 56,67,642,47
histo1 1
30/500 37,27,b27,67
histo1 1
31/500 57/5,643,57/5,665
histo1 1
32/500 4,5,b7,1
histo1 9
33/500 b3,b6,b57,1
histo1 1
34/500 16,4,47,4
histo1 2
35/500 56,46,5/6,4
histo1 1
36/500 5,6,364,4
histo1 1
37/500 164,7/6,6,2

histo1 1
291/500 16,67,L27,27
histo1 1
292/500 2,4,5/2,2
histo1 1
293/500 5/6,6,4,56/6
histo1 1
294/500 6,365,27,47
histo1 1
295/500 27,4,664,57/6
histo1 1
296/500 56/6,565/2,2,b7
histo1 1
297/500 7,37,57/2,26
histo1 1
298/500 b7,67,37,67
histo1 2
299/500 2,5,6,3
histo1 7
300/500 66,542/5,76/6,66
histo1 1
301/500 542,5,2,3
histo1 1
302/500 47,265,4,b66
histo1 1
303/500 57/5,565/5,57,565
histo1 1
304/500 642,67,47,5
histo1 1
305/500 57,7/6,67,4
histo1 1
306/500 b7,1,2,5
histo1 2
307/500 5,1,67,265
histo1 1
308/500 1,464,L242,665
histo1 1
309/500 5,5/6,4,5/6
histo1 1
310/500 56,1,4,27
histo1 1
311/500 3,26,2,26
histo1 1
312/500 4,164,76/5,56
histo1 1
313/500 57,365,57,164
histo1 1
314/500 5/2,4,1,5
histo1 1
315/500 16,27,47,56/6
histo1 1
316/500 6,36,b5,D164
histo1 1
317/500 4,1,4,1
histo2 220
318/500 3,46,56,464
histo1 1
319/500 27,57,17,47
histo1 5
320/500 56/6,6,b5,1
histo1 1
321/500 4,3,2,6
histo1 15
322/500 67,3,27,47
histo1 1
323/500 5,46,565,1
histo1 1
324/500 47,57/6,565/2,57/2
h

In [ ]:
##########################
# list errors here
##########################
45/500 56,46,164,4
2. Unable to extract for 56 46 164 4 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

237/500 4,16,5/5,5
2. Unable to extract for 4 16 5/5 5 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

381/500 3,6,27,5
2. Unable to extract for 3 6 27 5 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

478/500 564,1,56,46
2. Unable to extract for 564 1 56 46 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

In [5]:
##########################
# test errors here
##########################
mykey = '564,1,56,46'
page = 1
if page == 1:
    myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
else:
    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
print(myData.content)
#print(len(myData.json()))
#print(myData.json())

b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'


In [6]:
# ##########################
# # fix histo here
# ##########################
# histo[20] -= 4
# print(histo[:21])
# with open(histo_file, "w") as f:  
#     f.write(str(histo))

[0, 387, 48, 19, 8, 7, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 0]
